Import And Install Library

In [122]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import accuracy_score as acc
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

Unzip Dataset Kaggle Titanic Competion

In [2]:
!unzip titanic.zip -d titanic

Archive:  titanic.zip
replace titanic/gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Function

In [143]:
def fold(model):
  akurasi_kfold = []
  kf = KFold(n_splits=4,random_state=42,shuffle=True)
  for train_index, test_index in kf.split(x_train):

      X_train = x_train[train_index]
      Y_train = y_train[train_index]
      val = x_train[test_index]
      label_val = y_train[test_index]
      
      model.fit(X_train,Y_train)
      hasil = model.predict(val)
      akur = akurasi(hasil,label_val)
      akurasi_kfold.append(akur)
  akurasi_fix = sum(akurasi_kfold)/len(akurasi_kfold)
  return akurasi_fix

def akurasi(hasil,y_val):
    benar=0
    for i in range(len(hasil)):
        if hasil[i] == y_val[i]:
            benar+= 1
    akur = (benar/len(y_val))*100
    return akur

Get Data Train 

In [88]:
train = pd.read_csv("/content/titanic/train.csv")

train.columns.to_list()

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [89]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Data Cleaning

In [90]:
# Delete Unused Columns
delete_colums = ["PassengerId","Name", "SibSp","Parch", "Ticket", "Fare", "Cabin", "Embarked"]
train = train.drop(axis=1, labels=delete_colums)
train.head(10)

,Survived,Pclass,Sex,Age
0,0,3,male,22.0
1,1,1,female,38.0
2,1,3,female,26.0
3,1,1,female,35.0
4,0,3,male,35.0
5,0,3,male,NaN
6,0,1,male,54.0
7,0,3,male,2.0
8,1,3,female,27.0
9,1,2,female,14.0


In [91]:
# Fill Data NaN
train = train.fillna(method='ffill')

Data Exploration

In [92]:
train = train.replace("male", 0)
train = train.replace("female",1)
train["Age"] = train['Age'].astype('int')
train

,Survived,Pclass,Sex,Age
0,0,3,0,22
1,1,1,1,38
2,1,3,1,26
3,1,1,1,35
4,0,3,0,35
...,...,...,...,...
886,0,2,0,27
887,1,1,1,19
888,0,3,1,19
889,1,1,0,26


Build Model

In [ ]:
x_train = np.array(train.iloc[:, 1:])
y_train = np.array(train.iloc[:,0])

In [140]:
# linear
print('Linear')
c = [0.1,1]
for i in c:
    model= svm.SVC(kernel='linear',C = i)
    akur = fold(model)
    print("C : ", i , " akurasi : ",akur)

Linear
C :  0.1  akurasi :  78.67783702985497
C :  1  akurasi :  78.67783702985497


In [137]:
# Polinomial
d = [2,3,4,5,6]
print('poly')
for i in d :
    model= svm.SVC(kernel='poly', degree = i, gamma='scale')
    akur = fold(model)
    print("Degree : ", i , " akurasi : ", akur)

poly
Degree :  2  akurasi :  63.18830040803135
Degree :  3  akurasi :  66.21924615198158
Degree :  4  akurasi :  67.79178281420434
Degree :  5  akurasi :  67.56605259968488
Degree :  6  akurasi :  67.22922474043551


In [142]:
# Radial Basis Function
c,g = [0.1,1],[1,0.01,0.001,0.0001,0.00001]
print('rbf')
for i in c:
    temp = []
    for j in g:
        model= svm.SVC(kernel='rbf',C = i, gamma = j)
        akur = fold(model)
        print("C : ", i , " Gamma : ", j , " akurasi : ",akur)

rbf
C :  0.1  Gamma :  1  akurasi :  61.61778370298549
C :  0.1  Gamma :  0.01  akurasi :  62.17781683028319
C :  0.1  Gamma :  0.001  akurasi :  61.61778370298549
C :  0.1  Gamma :  0.0001  akurasi :  61.61778370298549
C :  0.1  Gamma :  1e-05  akurasi :  61.61778370298549
C :  1  Gamma :  1  akurasi :  78.89700238354946
C :  1  Gamma :  0.01  akurasi :  75.19038096392356
C :  1  Gamma :  0.001  akurasi :  62.62675231285097
C :  1  Gamma :  0.0001  akurasi :  61.61778370298549
C :  1  Gamma :  1e-05  akurasi :  61.61778370298549


Get Data Test And Class Test

In [158]:
test = pd.read_csv("/content/titanic/test.csv")
label_test = pd.read_csv("/content/titanic/gender_submission.csv")
test.columns.to_list()

['PassengerId',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [159]:
test.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [160]:
label_test.head(10)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


Data Test Cleaning

In [161]:
delete_colums = ["PassengerId","Name", "SibSp","Parch", "Ticket", "Fare", "Cabin", "Embarked"]
test = test.drop(axis=1, labels=delete_colums)
test

,Pclass,Sex,Age
0,3,male,34.5
1,3,female,47.0
2,2,male,62.0
3,3,male,27.0
4,3,female,22.0
...,...,...,...
413,3,male,NaN
414,1,female,39.0
415,3,male,38.5
416,3,male,NaN


In [162]:
# Fill Data NaN
test = test.fillna(method='ffill')
test = test.replace("male", 0)
test = test.replace("female",1)
test["Age"] = test['Age'].astype('int')
test

,Pclass,Sex,Age
0,3,0,34
1,3,1,47
2,2,0,62
3,3,0,27
4,3,1,22
...,...,...,...
413,3,0,28
414,1,1,39
415,3,0,38
416,3,0,38


In [198]:
model= svm.SVC(kernel='rbf',C = 1, gamma = 1)
model.fit(x_train,y_train)
x_test = np.array(test)
y_pred = model.predict(x_test)
y_test = np.array(label_test.iloc[:,1])
acc(y_test,y_pred) * 100

87.08133971291866

In [192]:
pd.DataFrame(y_test)

,0
0,0
1,1
2,0
3,0
4,1
...,...
413,0
414,1
415,0
416,0


In [193]:
pd.DataFrame(y_pred)

,0
0,0
1,0
2,1
3,0
4,1
...,...
413,0
414,1
415,0
416,0


In [196]:
data = pd.DataFrame([np.array(label_test.iloc[:,0]),y_pred]).T
data = np.array(data)
colum = ['PassengerId','Survived']
submission = pd.DataFrame(data, columns = colum )
submission
submission.to_csv('Submission_Rbf.csv', sep = ',', index=False,header = True)